In [39]:
#References are taken from Restaurant Review Project(https://www.geeksforgeeks.org/python-nlp-analysis-of-restaurant-reviews/)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [68]:
df = pd.read_excel('Women_nlp.xlsx')
df.head()#Orignial data set

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1


In [69]:
df.shape

(23488, 2)

In [70]:
df.isna().sum()  #We are getting 845 Review Text with NAN cluln , we need to remove the same

Review Text        845
Recommended IND      0
dtype: int64

In [87]:
#df1=df.dropna()#

df = df.dropna() 
df = df.reset_index() # reset's row indexes in case any rows were dropped
df.head()

#Drop row columns


,index,Review Text,Recommended IND
0,0,Absolutely wonderful - silky and sexy and comf...,1
1,1,Love this dress! it's sooo pretty. i happene...,1
2,2,I had such high hopes for this dress and reall...,0
3,3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,4,This shirt is very flattering to all due to th...,1


In [90]:
df.isna().sum() #no nan value in Reviews

index              0
Review Text        0
Recommended IND    0
dtype: int64

In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22643 entries, 0 to 22642
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            22643 non-null  int64 
 1   Review Text      22643 non-null  object
 2   Recommended IND  22643 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 530.8+ KB


In [92]:
df.shape

(22643, 3)

In [93]:
#Cleaninng Text
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 22643):
  review = re.sub('[^a-zA-Z]', ' ', df['Review Text'][i]) #Every symbol,quotes will replaced by space,(^a-zA-Z)-We will not replace letter with space.
  review = review.lower()                                   # convert all cases to lower cases 
  review = review.split()                                    # split to array(default delimiter is " ") 
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [94]:
print(corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [95]:
#Creating Bag of Models
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000)  #Max feature is nothing but number of imp words are categorized by model on which our model is being trained.Our model categorized 9000 words while we check max feature on 9000 workds we are getting poor accurcay of 40%,while we checked model on 1000 feature we get 73% accuracy.If we further reduce the max feature to 100 words we getting accuracy of 76%
X = cv.fit_transform(corpus).toarray()
y = df1.iloc[:, -1].values

In [96]:
#finding words in X(review Columns)

len(X[0])  #We got 9265 words taken from all the reviews with the help of tokenization, We will take 9000 words for our bag of words model.

1000

In [97]:
#Splitting model in train and test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [98]:
#Train Naives bayes model on Training Set

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)


GaussianNB(priors=None, var_smoothing=1e-09)

In [99]:
#Train set result,73 %accuracy on max feature of 1000,feel free to change max feature in naives bayes to check accuracy on another max feature


y_pred = classifier.predict(X_train)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_train.reshape(len(y_train),1)),1))

#Confusion matrix for Train Set
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_train, y_pred)
print(cm)
accuracy_score(y_train, y_pred)

[[0 0]
 [0 1]
 [1 0]
 ...
 [0 1]
 [1 1]
 [0 0]]
[[2858  386]
 [5013 9857]]


0.7019432483162195

In [100]:
#Test set result,73 %accuracy on max feature of 1000,feel free to change max feature in naives bayes to check accuracy on another max feature


y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1 1]
 [0 1]
 [1 1]
 ...
 [1 0]
 [1 1]
 [0 1]]
[[ 697  160]
 [1248 2424]]


0.6891145948332965

In [102]:
#This model/code is imported from this data set(https://www.geeksforgeeks.org/python-nlp-analysis-of-restaurant-reviews/)
#We are able to make 73% of correct prediction on our data set with 22k observations on which our model is trained

#Fitting a Predictive Model (here random forest)

#Since Random fored is ensemble model (made of many trees) from sklearn.ensemble, import RandomForestClassifier class
#With 501 tree or “n_estimators” and criterion as ‘entropy’
#Fit the model via .fit() method with attributes X_train and y_train
# Fitting Random Forest Classification 
# to the Training set 
from sklearn.ensemble import RandomForestClassifier 
  
# n_estimators can be said as number of 
# trees, experiment with n_estimators 
# to get better results  
model = RandomForestClassifier(n_estimators = 500, 
                            criterion = 'entropy') 
                              
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1 1]
 [0 1]
 [1 1]
 ...
 [1 0]
 [1 1]
 [0 1]]
[[ 697  160]
 [1248 2424]]


0.6891145948332965